In [135]:
import os
import cv2
import pandas as pd
import numpy as np
import PIL
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split

import torchvision.transforms as transforms
import torchvision.models as models
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

In [136]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [65]:
def dataloader(datapath='/Users/iyongbin/Repository/goormi-robot/data/', mode='train'):
    labels = {'angry':0, 'happy':1, 'neutral':2, 'sad':3, 'surprised':4}
    classes = []
    imgs = []
    for label in labels.keys():
        dir_path = os.path.join(datapath, mode, label)
        for img_name in os.listdir(dir_path):
            img_path = os.path.join(dir_path, img_name)
            img_pil = PIL.Image.open(img_path)
            img = img_pil.convert('RGB')
            img = transform(img)
            imgs.append(img)
            classes.append(labels[label])
    imgs = torch.stack(imgs)
    classes = torch.LongTensor(classes)

    return imgs, classes

In [66]:
x_train, y_train = dataloader(mode='train')
x_valid, y_valid = dataloader(mode='test')
print(x_train.shape, x_valid.shape)

torch.Size([24176, 3, 48, 48]) torch.Size([6043, 3, 48, 48])


In [67]:
batch_size = 128
epoches = 5
lr = 0.001

In [81]:
# dataset 
train_dataset = TensorDataset(x_train, y_train)
valid_dataset = TensorDataset(x_valid, y_valid)

# dataloader
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, drop_last=True)
valid_dataloader = DataLoader(valid_dataset, shuffle=False, batch_size=1, drop_last=True)

# Model


In [73]:
torch.cuda.is_available()

False

In [83]:
model = torch.load('/Users/iyongbin/Repository/goormi-robot/model/model_65.pth', map_location=torch.device('cpu'))

In [91]:
# pretrained resnet18
class resnet(nn.Module):
    def __init__(self, is_freezed=False):
        super(resnet, self).__init__()
        self.model = models.resnet18(weights='IMAGENET1K_V1')
        if is_freezed:
            for child in self.model.children():
                child.requires_grad = False
        self.model.fc = nn.Sequential(
            nn.Linear(self.model.fc.in_features, 256),
            nn.Linear(256, 5)
        )
    def forward(self, x):
        return self.model(x)

In [137]:
device = torch.device('cpu')
model = resnet(is_freezed=True)
model.load_state_dict(torch.load('/Users/iyongbin/Repository/goormi-robot/model/model_65.pth', map_location=device))

<All keys matched successfully>

In [129]:
def test_model(model, dataloader):
    y_test_pred = []
    model.eval()
    for X in dataloader:
        X = X[0]
        pred = model(X)
        
        y_test_pred.append(torch.argmax(pred, dim=1).item())
    return y_test_pred

In [139]:
import os
import cv2
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as tt


face_classifier = cv2.CascadeClassifier("/Users/iyongbin/Repository/goormi-robot/model/haarcascade_frontalface_default.xml")


def conv_block(in_channels, out_channels, pool=False):
    layers = [
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ELU(inplace=True),
    ]
    if pool:
        layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)


def conv_block(in_channels, out_channels, pool=False):
    layers = [
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True),
    ]
    if pool:
        layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)


In [160]:
labels = ['angry', 'happy', 'neutral', 'sad', 'surprised']

In [161]:
cap = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi_rgb = frame[y : y + h, x : x + w]
        roi_rgb = cv2.resize(roi_rgb, (48, 48), interpolation=cv2.INTER_AREA)
        print(roi_rgb.shape)

        if np.sum([roi_rgb]) != 0:
            roi = tt.functional.to_pil_image(roi_rgb)
            # roi = tt.functional.to_grayscale(roi)
            roi = tt.ToTensor()(roi).unsqueeze(0)

            # make a prediction on the ROI
            tensor = model(roi)
            pred = torch.max(tensor, dim=1)[1].tolist()
            print(pred[0])
            label = labels[pred[0]]

            label_position = (x, y)
            cv2.putText(
                frame,
                label,
                label_position,
                cv2.FONT_HERSHEY_COMPLEX,
                2,
                (0, 255, 0),
                3,
            )
        else:
            cv2.putText(
                frame,
                "No Face Found",
                (20, 60),
                cv2.FONT_HERSHEY_COMPLEX,
                2,
                (0, 255, 0),
                3,
            )

    cv2.imshow("Emotion Detector", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 48, 3)
1
(48, 4

KeyboardInterrupt: 

In [159]:
labels[1]

IndexError: list index out of range

In [130]:
test_dataset = TensorDataset(x_valid)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=1, drop_last=False)

In [131]:
y_test = test_model(model, test_dataloader)

In [132]:
y_test

[3,
 3,
 0,
 0,
 0,
 4,
 4,
 1,
 3,
 0,
 1,
 3,
 3,
 1,
 0,
 0,
 4,
 1,
 3,
 3,
 3,
 1,
 0,
 4,
 3,
 0,
 0,
 1,
 3,
 1,
 1,
 0,
 3,
 3,
 4,
 3,
 1,
 4,
 0,
 1,
 1,
 1,
 3,
 1,
 1,
 3,
 3,
 1,
 0,
 1,
 1,
 3,
 1,
 3,
 3,
 4,
 0,
 1,
 4,
 0,
 4,
 2,
 0,
 0,
 1,
 0,
 4,
 1,
 3,
 0,
 3,
 1,
 0,
 1,
 0,
 3,
 0,
 0,
 0,
 4,
 1,
 4,
 3,
 4,
 1,
 3,
 1,
 0,
 4,
 0,
 0,
 1,
 0,
 3,
 3,
 1,
 1,
 3,
 3,
 2,
 0,
 3,
 1,
 0,
 3,
 3,
 1,
 3,
 1,
 0,
 4,
 3,
 0,
 0,
 0,
 0,
 2,
 3,
 3,
 0,
 2,
 3,
 3,
 0,
 0,
 0,
 1,
 0,
 3,
 1,
 1,
 3,
 3,
 0,
 1,
 2,
 1,
 0,
 3,
 4,
 3,
 1,
 0,
 0,
 1,
 0,
 0,
 3,
 2,
 0,
 4,
 3,
 0,
 3,
 3,
 3,
 1,
 1,
 0,
 3,
 1,
 4,
 1,
 1,
 3,
 0,
 1,
 1,
 0,
 3,
 3,
 3,
 2,
 1,
 3,
 1,
 3,
 0,
 1,
 4,
 4,
 3,
 0,
 4,
 0,
 3,
 3,
 1,
 0,
 0,
 3,
 1,
 1,
 1,
 4,
 3,
 0,
 3,
 0,
 3,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 0,
 3,
 3,
 4,
 1,
 3,
 2,
 0,
 3,
 2,
 3,
 0,
 3,
 1,
 1,
 0,
 1,
 3,
 1,
 1,
 3,
 1,
 0,
 0,
 4,
 3,
 0,
 3,
 1,
 3,
 0,
 1,
 0,
 0,
 3,
 1,
 0,
 3,
 3,
 4,
 3,
 0,
 1,


In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if result.multi_face_landmarks:
        for landmarks in result.multi_face_landmarks:
            for point in landmarks.landmark:
                height, width, _ = frame.shape
                cx, cy = int(point.x * width), int(point.y * height)
                cv2.circle(frame, (cx, cy), 2, (0, 255, 0), -1)

    # 결과를 화면에 표시
    cv2.imshow("Face Landmarks", frame)

    if cv2.waitKey(1) & 0xFF == 27:  # 'ESC' 키를 누르면 종료
        break

# 종료 시 자원 해제
cap.release()
cv2.destroyAllWindows()
